In [1]:
import torch
from rich.markdown import Markdown

device = ('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [2]:
%cd ..

/home/oscarn/flan-gpt2


In [3]:
from models.model_utils import GPT2Model

wrapped_model = GPT2Model("output/gpt2_med_lora-distill_6000-flan_t5_xl_qntz", device, peft=True)
peft_model = wrapped_model.get_model(peft=True)
peft_tokenizer = wrapped_model.get_tokenizer()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [4]:
from eval.eval_utils import Evaluation
from data.data_utils import create_instruct_dataset


eval = Evaluation(peft_model, peft_tokenizer, device)

2025-09-16 17:28:21,421 - datasets - INFO - PyTorch version 2.5.1 available.


In [5]:
num_samples = 100
CAUSAL_LM = True

In [6]:
template = """
## INPUT PROMPT
{prompt}
## GROUND_TRUTH
{completion}
## MODEL OUTPUT
{prediction}
"""

### ANLI

In [7]:
dataset = create_instruct_dataset(num_samples, ["anli"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/16946 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16946 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.59it/s]


INPUT PROMPT                                                    

Determine if the sentence is true based on the text below: Trust Company Ltd. was founded in 2002.                 

Trust Company Ltd. (株式会社トラスト , Kabushikigaisha Torasuto ) is a Japanese automotive aftermarket company     
specialising in performance tuning parts for cars. The company is widely known for its subbrand of tuning parts    
GReddy and the turbochargers under this brand. OPTIONS:                                                            

 • entailment                                                                                                      
 • neutral                                                                                                         
 • contradiction                                                                                                   


                                                   GROUND_TRUTH                                                    

neutral                                                                                                            


                                                   MODEL OUTPUT                                                    

neutral

## Common Gen

In [8]:
dataset = create_instruct_dataset(num_samples, ["common_gen"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  2.96it/s]


INPUT PROMPT                                                    

Generate a sentence that includes all the following words: ['boat', 'sit', 'beach']                                


                                                   GROUND_TRUTH                                                    

Pelicans sit on a blue boat at the beach.                                                                          


                                                   MODEL OUTPUT                                                    

boat sits on a beach

## SQUAD

In [10]:
dataset = create_instruct_dataset(num_samples, ["squad"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
2025-09-16 17:29:27,815 - datasets.load - WARNING - Using the latest cached version of the dataset since rajpurkar/squad couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).
2025-09-16 17:29:27,822 - datasets.packaged_modules.cache.cache - WARNING - Found the latest cached dataset configuration 'plain_text' at /home/oscarn/.cache/huggingface/datasets/rajpurkar___squad/plain_text/0.0.0/7b6d24c440a36b6815f21b70d25016731768db1f (last modified on Fri Nov 22 00:36:36 2024).


Filter:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/87241 [00:00<?, ? examples/s]

Filter:   0%|          | 0/87241 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  7.67it/s]


INPUT PROMPT                                                    

Read this and answer the question                                                                                  

About 80% of undergraduates and 20% of graduate students live on campus. The majority of the graduate students on  
campus live in one of four graduate housing complexes on campus, while all on-campus undergraduates live in one of 
the 29 residence halls. Because of the religious affiliation of the university, all residence halls are single-sex,
with 15 male dorms and 14 female dorms. The university maintains a visiting policy (known as parietal hours) for   
those students who live in dormitories, specifying times when members of the opposite sex are allowed to visit     
other students' dorm rooms; however, all residence halls have 24-hour social spaces for students regardless of     
gender. Many residence halls have at least one nun and/or priest as a resident. There are no traditional social    
fraternities or sororities at the university, but a majority of students live in the same residence hall for all   
four years. Some intramural sports are based on residence hall teams, where the university offers the only         
non-military academy program of full-contact intramural American football. At the end of the intramural season, the
championship game is played on the field in Notre Dame Stadium.                                                    

There are how many dorms for females at Notre Dame?                                                                


                                                   GROUND_TRUTH                                                    

14                                                                                                                 


                                                   MODEL OUTPUT                                                    

40

## Cosmos QA

In [11]:
dataset = create_instruct_dataset(num_samples, ["cosmos_qa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.
2025-09-16 17:29:54,484 - datasets.load - WARNING - Using the latest cached version of the module from /home/oscarn/.cache/huggingface/modules/datasets_modules/datasets/allenai--cosmos_qa/3e18538cbfdb2c04189b16642715f0f6da3e97ed5df0aadcec3641245b2cf157 (last modified on Fri Nov 22 00:37:03 2024) since it couldn't be found locally at allenai/cosmos_qa, or remotely on the Hugging Face Hub.


Map:   0%|          | 0/25262 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25262 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.62it/s]


INPUT PROMPT                                                    

Context:I plan on doing some more work next weekend , by which time I 'll have regained my strength . It really is 
the most satisfying work ; I used to do chores like this all the time growing up , when I had a huge backyard and  
vegetable garden . Some of the neighbors would let me trim their trees when I asked , and I 'd climb around in them
with a hatchet and a hand saw , cutting off the branches that were growing too close to windows and power lines . I
'd rake leaves and pull weeds for dollars . Question What will happen after they are done cleaning the garden and  
trimming trees ? Answer: OPTIONS:                                                                                  

 • They will fall out of the tree when in it cutting branches .                                                    
 • Their yard will stay nice and look great .                                                                      
 • They will be to tired to keep up the work .                                                                     
 • None of the above choices .                                                                                     


                                                   GROUND_TRUTH                                                    

Their yard will stay nice and look great .                                                                         


                                                   MODEL OUTPUT                                                    

They will be to tired to keep up the work.

## CoQA

In [12]:
dataset = create_instruct_dataset(num_samples, ["coqa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=150, return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/7199 [00:00<?, ? examples/s]

Map:   0%|          | 0/4063 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4063 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:04<00:00,  4.65s/it]


INPUT PROMPT                                                    

I'm Larry. I'm really good at selling things. I also love helping people. But I'm not so good at solving problems. 
I think I'd like to be a salesman or detective . I'm Anita. I really like doing things with my hands. I also enjoy 
working with wood. I don't enjoy working in the same place every day, and I hate being in noisy places. I think I'd
like to be a factory worker or a carpenter . I'm Jill. I'm good at explaining things and I really like children. I 
can't stand working long hours. I think I'd like to be a doctor or a teacher. I'm Maria. I'm really interested in  
meeting people, and I enjoy wearing different clothes every day. I'm not so good at organizing my time and I can't 
stand computers. I am going to be a model. I'm Jim. I enjoy helping people, but I can't stand working nights and   
weekends. I want to be a nurse or a social worker.                                                                 

  1 What are they good at?                                                                                         
  2 What is their name?                                                                                            
  3 What do they like to do?                                                                                       
  4 Is there anything they aren't good at?                                                                         
  5 What is it?                                                                                                    
  6 What do they want to be?                                                                                       
  7 Who would enjoy carving?                                                                                       
  8 What does she like to use?                                                                                     
  9 What doesn't she enjoy?                                                                                        
 10 What else?                                                                                                     
 11 What would they like as a job?                                                                                 
 12 Or?                                                                                                            
 13 Who wants to be a physician?                                                                                   
 14 What does she not like to do?                                                                                  
 15 And the opposite?                                                                                              
 16 Who likes fashion?                                                                                             
 17 What in particular?                                                                                            
 18 What does she dislike?                                                                                         
 19 What does another boy want to be?                                                                              
 20 What is his name?                                                                                              

Provide a numbered list of answers.                                                                                


                                                   GROUND_TRUTH                                                    

  1 selling things                                                                                                 
  2 Larry                                                                                                          
  3 help people                                                                                                    
  4 I'm not so good at solving problems                                                              

## XSum

In [13]:
dataset = create_instruct_dataset(num_samples, ["xsum"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Using the latest cached version of the dataset since EdinburghNLP/xsum couldn't be found on the Hugging Face Hub
2025-09-16 17:31:08,800 - datasets.load - WARNING - Using the latest cached version of the dataset since EdinburghNLP/xsum couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/oscarn/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/40db7604fedb616a9d2b0673d11838fa5be8451c (last modified on Tue Sep 16 05:49:23 2025).
2025-09-16 17:31:08,807 - datasets.packaged_modules.cache.cache - WARNING - Found the latest cached dataset configuration 'default' at /home/oscarn/.cache/huggingface/datasets/EdinburghNLP___xsum/default/1.2.0/40db7604fedb616a9d2b0673d11838fa5be8451c (last modified on Tue Sep 16 05:49:23 2025).


Filter:   0%|          | 0/204045 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:01<00:00,  1.71s/it]


INPUT PROMPT                                                    

Gundogan, 26, told BBC Sport he "can see the finishing line" after tearing cruciate knee ligaments in December, but
will not rush his return. The German missed the 2014 World Cup following back surgery that kept him out for a year,
and sat out Euro 2016 because of a dislocated kneecap. He said: "It is heavy mentally to accept that." Gundogan    
will not be fit for the start of the Premier League season at Brighton on 12 August but said his recovery time is  
now being measured in "weeks" rather than months. He told BBC Sport: "It is really hard always to fall and fight   
your way back. You feel good and feel ready, then you get the next kick. "The worst part is behind me now. I want  
to feel ready when I am fully back. I want to feel safe and confident. I don't mind if it is two weeks or six."    
Gundogan made 15 appearances and scored five goals in his debut season for City following his £20m move from       
Borussia Dortmund. He is eager to get on the field again and was impressed at the club's 4-1 win over Real Madrid  
in a pre-season game in Los Angeles on Wednesday. Manager Pep Guardiola has made five new signings already this    
summer and continues to have an interest in Arsenal forward Alexis Sanchez and Monaco's Kylian Mbappe. Gundogan    
said: "Optimism for the season is big. It is huge, definitely. "We felt that last year as well but it was a        
completely new experience for all of us. We know the Premier League a bit more now and can't wait for the season to
start." City complete their three-match tour of the United States against Tottenham in Nashville on Saturday.      
Chelsea manager Antonio Conte said earlier this week he did not feel Tottenham were judged by the same standards as
his own side, City and Manchester United. Spurs have had the advantage in their recent meetings with City, winning 
three and drawing one of their last four Premier League games. And Gundogan thinks they are a major threat. He     
said: "Tottenham are a great team. They have the style of football. They have young English players. Our experience
last season shows it is really tough to beat them. "They are really uncomfortable to play against. "I am pretty    
sure, even if they will not say it loud, the people who know the Premier League know Tottenham are definitely a    
competitor for the title." What was that article about?                                                            


                                                   GROUND_TRUTH                                                    

Manchester City midfielder Ilkay Gundogan says it has been mentally tough to overcome a third major injury.        


                                                   MODEL OUTPUT                                                    

Manchester United coach Antonio Conte has admitted that Chelsea's defeat of Arsenal in the Champions League final  
on away goals this season is a "scaffolding".

## BoolQ

In [14]:
dataset = create_instruct_dataset(num_samples, ["bool_q"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/9427 [00:00<?, ? examples/s]

Map:   0%|          | 0/9399 [00:00<?, ? examples/s]

Filter:   0%|          | 0/9399 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  8.69it/s]


INPUT PROMPT                                                    

Unbreakable glass is glass, or glass substitute, which does not display the normal fragility of glass - in general 
the term is not used to refer to something that is absolutely unbreakable.                                         

What's the best answer to this question: is there such a thing as unbreakable glass?                               

OPTIONS:                                                                                                           

 • True                                                                                                            
 • False                                                                                                           


                                                   GROUND_TRUTH                                                    

True                                                                                                               


                                                   MODEL OUTPUT                                                    

False

## Python Code

In [16]:
dataset = create_instruct_dataset(num_samples, ["python_code"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], max_tokens=1000, return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/16750 [00:00<?, ? examples/s]

Map:   0%|          | 0/16237 [00:00<?, ? examples/s]

Filter:   0%|          | 0/16237 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:20<00:00, 20.08s/it]


INPUT PROMPT                                                    

Write a python program to extract the non-vowel words in a sentence. This machine learning project is challenging. 


                                                   GROUND_TRUTH                                                    

                                                                                                                   
 def extract_words(sentence):                                                                                      
     non_vowel_words = []                                                                                          
     words = sentence.split()                                                                                      
     vowels = ['a','e','i','o','u','A','E','I','O','U']                                                            
     for word in words:                                                                                            
         if not any(char in vowels for char in word):                                                              
             non_vowel_words.append(word)                                                                          
     return non_vowel_words                                                                                        
                                                                                                                   


                                                   MODEL OUTPUT                                                    

                                                                                                                   
 def extract_words:                                                                                                
     self.vocab = ''                                                                                               
       words = self.vocab.split()                                                                                  
      if words == nil:                                                                                             
         return None                                                                                               
         elif words.startswith('*').startswith('-') and words.endswith('!').startswith('-') and                    
 words.startswith('-'):                                                                                            
            words.append(words.pop(0, len(words)) + words.pop(0, len(words)) )                                     
       return words                                                                                                
                                                                                                                   
 labels = ['Vocab', 'Pronation']                                                                                   
 languages = ['computer']                                                                                          
                                                                                                                   
 for language in language:                                                                                         
        lg = extract_words()                                                                                       
                                                                                                                   
 for language in labels:                                                                                           
          while True:                         

## Eng-Spa

In [19]:
dataset = create_instruct_dataset(num_samples, ["eng_spa"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.19it/s]


INPUT PROMPT                                                    

Translate from Spanish to English:                                                                                 

Ocurre con más frecuencia de lo que crees.                                                                         


                                                   GROUND_TRUTH                                                    

It happens more often than you would think.                                                                        


                                                   MODEL OUTPUT                                                    

I can't drive for you this day.

## PAWS

In [20]:
dataset = create_instruct_dataset(num_samples, ["paws"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Map:   0%|          | 0/49401 [00:00<?, ? examples/s]

Filter:   0%|          | 0/49401 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  8.86it/s]


INPUT PROMPT                                                    

Do the following two sentences have the same meaning? These include replicas at Ramat Shlomo in Israel and in Kfar 
Chabad in Jerusalem . These include replicas in Ramat Shlomo in Jerusalem and in Kfar Chabad , Israel .            

OPTIONS:                                                                                                           

 • No                                                                                                              
 • Yes                                                                                                             


                                                   GROUND_TRUTH                                                    

No                                                                                                                 


                                                   MODEL OUTPUT                                                    

No

## Quora

In [23]:
dataset = create_instruct_dataset(num_samples, ["quora"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/56402 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:00<00:00,  1.37it/s]


INPUT PROMPT                                                    

If an atheist doesn't believe in God, do they also not believe in Satan?                                           


                                                   GROUND_TRUTH                                                    

AN ATHEIST, BY DEFINITION DOESN’T BELIEVE IN ANYTHING HE/SHE CAN’T SEE.                                            


                                                   MODEL OUTPUT                                                    

OPTIONS:                                                                                                           

 • True                                                                                                            
 • FalseTrue

## Alpaca

In [24]:
dataset = create_instruct_dataset(num_samples, ["alpaca"])
example = dataset[0]
prediction = eval.generate([example["prompt"]], return_full_text=not CAUSAL_LM)[0]
Markdown(template.format(prompt=example["prompt"], 
                         completion=example["completion"], 
                         prediction=prediction))

Filter:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/20100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Generating responses... : 100%|██████████| 1/1 [00:02<00:00,  2.06s/it]


INPUT PROMPT                                                    

Arrange the given numbers in ascending order. 2, 4, 0, 8, 3                                                        


                                                   GROUND_TRUTH                                                    

0, 2, 3, 4, 8                                                                                                      


                                                   MODEL OUTPUT                                                    

, 20, 4```python for num in range(2, 4, 0): print(num, ": " + str(num))                                            

                                                                                                                   
                                                                                                                   
                                                                                                                   
